# AoE2 Replay Analyzer — RECSAGE
Métricas clave por jugador: aldeanos creados, tiempo de TC inactivo (idle) y APM.

Funciona en local y en Google Colab. Si usas Colab, ejecuta primero la celda de instalación.


In [ ]:
# %% (Opcional en Colab) Instalar dependencias
# Ejecuta en Colab si falta 'mgz':
# !pip -q install --upgrade pip wheel
# !pip -q install mgz pandas numpy matplotlib tqdm


In [ ]:
# %% Imports y utilidades
import re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Librería mgz (pura-Python)
from mgz.model import parse_match  # devuelve objeto Match

WINDOW_SEC = 60  # ventana para APM
_VILLAGER_RE = re.compile(r'villager|aldean', re.IGNORECASE)

def load_match(replay_path: str):
    with open(replay_path, 'rb') as fh:
        return parse_match(fh)

def is_villager(unit_name: str | None) -> bool:
    if not unit_name:
        return False
    return bool(_VILLAGER_RE.search(unit_name))

def villager_counts(match):
    counts = {p.number: 0 for p in match.players}
    for act in match.actions:
        if act.type.name != 'TRAIN':
            continue
        unit_obj = act.payload.get('unit') or {}
        name = (getattr(unit_obj, 'name', None) or
                getattr(unit_obj, 'unit_name', None) or
                (unit_obj.get('name') if isinstance(unit_obj, dict) else None) or
                act.payload.get('unit_name'))
        if is_villager(name):
            pid = act.player.number if act.player else None
            if pid is not None:
                counts[pid] += 1
    return counts

def tc_idle_time(match, base_prod_time: float = 25.0, gap_threshold: float = 27.0):
    """Aproximación: tiempo TC inactivo = (gap entre aldeanos) - base_prod_time
    si el gap supera 'gap_threshold'.
    """
    idle = {p.number: 0.0 for p in match.players}
    last_train = {p.number: None for p in match.players}
    for act in match.actions:
        if act.type.name != 'TRAIN':
            continue
        unit_obj = act.payload.get('unit') or {}
        name = (getattr(unit_obj, 'name', None) or
                getattr(unit_obj, 'unit_name', None) or
                (unit_obj.get('name') if isinstance(unit_obj, dict) else None) or
                act.payload.get('unit_name'))
        if not is_villager(name):
            continue
        pid = act.player.number if act.player else None
        if pid is None:
            continue
        t = act.timestamp.total_seconds()
        if last_train[pid] is not None:
            gap = t - last_train[pid]
            if gap > gap_threshold:
                idle[pid] += max(0.0, gap - base_prod_time)
        last_train[pid] = t
    return idle

def apm_timeseries(match, window_sec: int = WINDOW_SEC):
    rows = [(act.timestamp.total_seconds(), act.player.number)
            for act in match.actions if act.player]
    if not rows:
        return pd.DataFrame()
    df = pd.DataFrame(rows, columns=['t', 'player'])
    max_t = df['t'].max()
    bins = np.arange(0, max_t + window_sec, window_sec)
    apm = {}
    for pid in df['player'].unique():
        counts, _ = np.histogram(df.loc[df['player'] == pid, 't'], bins=bins)
        apm[pid] = counts * 60 / window_sec
    ts = pd.DataFrame(apm, index=bins[:-1])
    ts.index.name = 'time_sec'
    return ts

def plot_apm(ts, match):
    if ts.empty:
        print('Sin acciones suficientes para APM.')
        return
    plt.figure(figsize=(10, 6))
    for pid in ts.columns:
        name = next(p.name for p in match.players if p.number == pid)
        plt.plot(ts.index / 60, ts[pid], label=name)
    plt.xlabel('Tiempo (min)')
    plt.ylabel('APM')
    plt.title(f'APM por jugador — ventana {WINDOW_SEC}s')
    plt.grid(True)
    plt.legend()
    plt.show()

def plot_apm_bar(ts, match):
    if ts.empty:
        print('Sin datos para generar barplot de APM.')
        return
    means = ts.mean()
    stds = ts.std()
    names = [next(p.name for p in match.players if p.number == pid) for pid in means.index]
    x = np.arange(len(names))
    plt.figure(figsize=(6, 5))
    plt.bar(x, means.values, yerr=stds.values, capsize=6)
    plt.xticks(x, names, rotation=45, ha='right')
    plt.ylabel('APM medio')
    plt.title('APM medio ± desviación estándar')
    plt.tight_layout()
    plt.show()


In [ ]:
# %% Selección del replay (Colab o local)
REPLAY_PATH = None
try:
    from google.colab import files  # type: ignore
    print('🔄 Sube un archivo .aoe2record…')
    uploaded = files.upload()
    if uploaded:
        REPLAY_PATH = next(iter(uploaded))
except Exception:
    pass

if REPLAY_PATH is None:
    default_path = Path('AOE2_STATPARSER/AgeIIDE_Replay_396581946.aoe2record')
    if default_path.exists():
        REPLAY_PATH = str(default_path)
    else:
        raise RuntimeError('No se ha seleccionado replay. Sube un .aoe2record o ajusta REPLAY_PATH.')

match = load_match(REPLAY_PATH)
print(f'Mapa: {match.map.name} — Duración: {match.duration.total_seconds()/60:.1f} min')


In [ ]:
# %% Cálculo de métricas y resumen por jugador
villagers = villager_counts(match)
idles = tc_idle_time(match)
aps = apm_timeseries(match)

rows = []
for p in match.players:
    pid = p.number
    name = p.name
    civ = getattr(p, 'civilization', None)
    apm_mean = float(aps[pid].mean()) if (not aps.empty and pid in aps) else np.nan
    apm_peak = float(aps[pid].max()) if (not aps.empty and pid in aps) else np.nan
    idle_s = float(idles.get(pid, 0.0))
    dur_s = match.duration.total_seconds()
    idle_pct = 100.0 * idle_s / dur_s if dur_s > 0 else np.nan
    rows.append({
        'player': name,
        'civ': civ,
        'villagers_trained': int(villagers.get(pid, 0)),
        'tc_idle_s': round(idle_s, 1),
        'tc_idle_%': round(idle_pct, 1),
        'apm_mean': round(apm_mean, 1) if apm_mean == apm_mean else np.nan,
        'apm_peak': round(apm_peak, 1) if apm_peak == apm_peak else np.nan,
    })

summary = pd.DataFrame(rows).set_index('player')
summary


In [ ]:
# %% Visualizaciones APM
plot_apm(aps, match)
plot_apm_bar(aps, match)
